# Precision and speed of fitting a ridge regressions in overparametrized regime

The goal of this notebook is to compare the performance (precision and speed) of different implementations of ridge regression in the overparametrized regime. 

The main issues are ...

Here we compare: 
- original method from VOC
- scikit learn implementations of ridge regression
- ??? other



In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import json

from pyinstrument import Profiler
%load_ext pyinstrument

from sklearn import set_config
set_config(assume_finite=True) 

import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
import datetime as dt

# Model fitting

[VoC] (regularized) least-squares estimators of the form

$$ \hat{\beta}(z) = \left( zI + T^{-1} \sum_t S_t S_t' \right)^{-1} \frac{1}{T} \sum_t S_t R_{t+1}$$

The ridge regression in The Elements of Statistical Learning (Hastie et al., 2009) is equivalent to the above equation with $z T = \lambda $ and given as 

$$ \hat{\beta}(\lambda) = \left( \lambda I + X^T X \right)^{-1}X^Ty$$

In scikit-learn parameter $\lambda$ is labeled with $\alpha$.

In [55]:
def single_run(run_inputs, run_params, delta_t=1):
    """
    Run the backtest for a single run.
    S: matrix of features
    R: vector of target variable, shifted once 
    T_list: list of training window lengths
    model_list: list of models to train (ridge regressions with different lambdas)
    P_list: list of number of features to use for each model
    delta_t: time step for retraining the model

    Returns:
    """

    S, R = run_inputs
    T_list, P_list, model_dict = run_params

    # initialize arrays for storing results
    num_of_models = len(model_dict[T_list[0]])

    min_T = min(T_list) # usually =12

    # dimensions: (ts, Ts, Ps, lambdas)
    output_shape = (len(S)-min_T, len(T_list), len(P_list), num_of_models)
    beta_norm_sq = np.full(shape=output_shape, 
                    fill_value=np.nan, 
                    dtype=np.float64)
    return_forecasts = np.full(shape=output_shape, 
                               fill_value=np.nan, 
                               dtype=np.float64)
    strategy_returns = np.full(shape=output_shape, 
                               fill_value=np.nan, 
                               dtype=np.float64)

    # initialize arrays for storing intermediate variables
    # training_std = np.full((len(T_arr), S.shape[1]), fill_value=np.nan, dtype=np.float64)

    def my_std(x):
        return np.sqrt(np.sum(np.square(x - x.mean(axis=0)))/len(x))

    def standardize(t):
        training_sets = []

        for T in T_list:

            # this take a lot of time to compute
            # train_std = trainX.std(axis=0) 
            # train_std = np.sqrt(np.sum((trainX - train_mean)**2, axis=0)/T) # this is equivalent to trainX.std(axis=0)
            if t-T < 1:
                training_sets.append((np.nan, np.nan))
            else:
                training_std = my_std(S[t-T:t])
                trainX = S[t-T:t] / training_std
                forecastX = S[t] / training_std
                training_sets.append((trainX, forecastX))

        return training_sets

    grid = [(P_index, model_index) 
            for P_index in range(len(P_list)) 
            for model_index in range(num_of_models)]
    

    for t in range(min_T, len(S), delta_t):
        # print progress
        if t%100==0:
            print(f"progress: {t/(len(S)-min_T):2.1%}")

        # one standardization for all models, all complexities
        training_sets = standardize(t)

        for P_index, model_index in grid:
            for T_index in range(len(T_list)):
                T = T_list[T_index]

                if t-T < 1:
                    continue
                else: 
                    P = P_list[P_index]
                    # get model of appropriate shrinkage lambda=T*z, z is tracked by model_index
                    model = model_dict[T][model_index]
                    
                    trainX, forecastX = training_sets[T_index]
                    trainY = R[t-T:t]
                    # take first P features for training
                    model.fit(trainX[:,:P], trainY) 
                    forecastY = model.predict(forecastX[:P].reshape(1,-1))

                    # store results 
                    beta_norm_sq[t-min_T, T_index, P_index, model_index] = np.sum(np.square(model.coef_))
                    return_forecasts[t-min_T, T_index, P_index, model_index] = forecastY[0]
                    strategy_returns[t-min_T, T_index, P_index, model_index] = forecastY[0] * R[t]

    return beta_norm_sq, return_forecasts, strategy_returns

In [56]:
# test run
T_list = [60, 120]
z_list = [0.01, 100]
model_dict = {T: [Ridge(alpha=T*z, solver="auto", fit_intercept = False) for z in z_list] for T in T_list}
P_list = [32768, 12000]
delta_t=1

run_inputs = (S, R)
run_params = (T_list, P_list, model_dict)

In [57]:
%%pyinstrument
b, r, sr = single_run(run_inputs, run_params, delta_t)

progress: 9.5%


KeyboardInterrupt: 

_     ._   __/__   _ _  _  _ _/_   Recorded: 00:48:10  Samples:  2376
 /_//_/// /_\ / //_// / //_'/ //     Duration: 8.386     CPU time: 57.499
/   _/                      v4.6.2

Program: /home/miroslav/miniforge3/envs/voc/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/miroslav/.local/share/jupyter/runtime/kernel-v2-2018470dTDUU1Ktkm3c.json

8.354 <module>  ../../../../../../../tmp/ipykernel_2019003/4008075110.py:1
`- 8.354 single_run  ../../../../../../../tmp/ipykernel_2019003/871893489.py:1
   |- 6.247 wrapper  sklearn/base.py:1457
   |     [16 frames hidden]  sklearn, <built-in>
   |        4.628 safe_sparse_dot  sklearn/utils/extmath.py:164
   |        `- 4.609 [self]  sklearn/utils/extmath.py
   |- 1.196 standardize  ../../../../../../../tmp/ipykernel_2019003/871893489.py:40
   |  |- 0.853 my_std  ../../../../../../../tmp/ipykernel_2019003/871893489.py:37
   |  |  |- 0.576 [self]  ../../../../../../../tmp/ipykernel_2019003/871893489.py
   |  |  `- 0.204 _mean  numpy/core/_methods.py:101
   |  |        [2 frames hidden]  numpy, <built-in>
   |  `- 0.344 [self]  ../../../../../../../tmp/ipykernel_2019003/871893489.py
   `- 0.875 Ridge.predict  sklearn/linear_model/_base.py:272
         [3 frames hidden]  sklearn

### 2.1 Profiling

50s for 8 points in 2x2x2 P-z-T grid, i.e. 6.5s per node.

Goal:
- $P \in [2, 8, 32, 128, 512, 2048, 8192, 32768, 12000]$
- $log10(z) \in [−3,−2,−1, 0, 1, 2, 3] $
- $T \in [12, 60, 120]$

Total runs: 9 * 7 * 3 = 189 nodes \
Total time: 189 * 6.5s = 21 minutes

It should be repeated up to 1000 times for low P (and low T?), and results should be averaged. For big P we can perform smaller number of repetitions.

Bottle necks:
- Ridge fitting wastes time on standardization and checking input data

Solvers and their performance:
solver="auto" - 50-70s depending on caching /
solver="lsqr" - 85s /
solver="sag" - way too slow


In [ ]:
# full parameter set

T_list = [12, 60, 120]
z_list = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
model_dict = {T: [Ridge(alpha=T*z, fit_intercept = False) for z in z_list] for T in T_list}
P_list = [2**i for i in range(1, 14)]
P_list.append(12000)
delta_t=1

run_inputs = (S, R)
run_params = (T_list, P_list, model_dict)

### Manual fitting

$$ \left( \lambda I + X^T X \right) \hat{\beta}(z) = X^Ty$$

$$ \left( \lambda I + S^T S \right) \hat{\beta}(z) = S^TR$$

The idea is to try to speedup the process of fitting the Ridge regression by manually solving the system of linear equations. This way, we would save time on operations done by the Ridge solver, such as standardization and checking the input data. **No speedup is achieved**, so this part of code is saved in case it is needed in the future.

In [ ]:
from scipy.linalg import lstsq
from numpy.linalg import lstsq as np_lstsq

def single_run_lsqr(S, R, T_list, model_dict, P_list, delta_t=1):
    """
    Run the backtest for a single run.
    S: matrix of features
    R: vector of target variable, shifted once 
    T_list: list of training window lengths
    model_list: list of models to train (ridge regressions with different lambdas)
    P_list: list of number of features to use for each model
    delta_t: time step for retraining the model

    Returns:
    """

    # initialize arrays for storing results
    num_of_models = len(model_dict[T_list[0]])

    # dimensions: (ts, Ts, Ps, lambdas)
    output_shape = (len(S)-min(T_list), len(T_list), len(P_list), num_of_models)
    beta_norm_sq = np.full(shape=output_shape, 
                    fill_value=np.nan, 
                    dtype=np.float64)
    return_forecasts = np.full(shape=output_shape, 
                               fill_value=np.nan, 
                               dtype=np.float64)
    strategy_returns = np.full(shape=output_shape, 
                               fill_value=np.nan, 
                               dtype=np.float64)

    # initialize arrays for storing intermediate variables
    # training_std = np.full((len(T_arr), S.shape[1]), fill_value=np.nan, dtype=np.float64)

    def my_std(x):
        return np.sqrt(np.sum(np.square(x - x.mean(axis=0)))/len(x))

    def standardize(t):
        training_sets = []

        for T in T_list:

            # this take a lot of time to compute
            # train_std = trainX.std(axis=0) 
            # train_std = np.sqrt(np.sum((trainX - train_mean)**2, axis=0)/T) # this is equivalent to trainX.std(axis=0)
            if t-T < 1:
                training_sets.append((np.nan, np.nan))
            else:
                training_std = my_std(S[t-T:t])
                trainX = S[t-T:t] / training_std
                forecastX = S[t] / training_std
                training_sets.append((trainX, forecastX))

        return training_sets

    grid = [(P_index, model_index) 
            for P_index in range(len(P_list)) 
            for model_index in range(num_of_models)]
    
    for t in range(min(T_list), len(S), delta_t):
        # print progress
        if t%100==0:
            print(f"progress [%]: {100*t/(len(S)-min(T_list)):.2f}")

        # one standardization for all models, all complexities
        training_sets = standardize(t)

        for P_index, model_index in grid:
            for i in range(len(T_list)):
                T = T_list[i]

                if t-T < 1:
                    continue
                else: 
                    P = P_list[P_index]
                    # get model of appropriate shrinkage lambda=T*z, z is tracked by model_index
                    lambda_param = model_dict[T][model_index]

                    
                    trainX, forecastX = training_sets[i]
                    trainY = R[t-T:t]
                    # take first P features for training

                    X = trainX[:,:P]
                    y = trainY

                    # model.fit(trainX[:,:P], trainY) 
                    LHS = lambda_param*np.eye(X.shape[1]) + X.T @ X
                    RHS = X.T @ y
                    # beta = lsqr(LHS, RHS, damp=np.sqrt(lambda_param), calc_var=False, x0=None)[0] # atol=1e-12, btol=1e-12, conlim=1e8, iter_lim=None, show=False, 
                    beta = lstsq(LHS, RHS, check_finite=False)[0] # atol=1e-12, btol=1e-12, conlim=1e8, iter_lim=None, show=False, 
                    # , lapack_driver='gelsy'
                    beta = np_lstsq(LHS, RHS)[0] # atol=1e-12, btol=1e-12, conlim=1e8, iter_lim=None, show=False,

                    # forecastY = model.predict(forecastX[:P].reshape(1,-1))
                    forecastY = beta @ forecastX[:P]

                    # store results 
                    beta_norm_sq[t-min(T_list), i, P_index, model_index] = np.sum(np.square(beta))
                    return_forecasts[t-min(T_list), i, P_index, model_index] = forecastY
                    strategy_returns[t-min(T_list), i, P_index, model_index] = forecastY * R[t]
                    print(type(R))
                    print(type(R[t]))
    return beta_norm_sq, return_forecasts, strategy_returns

In [ ]:
# test run
# T_list = [60, 120]
# z_list = [0.01, 100]
# model_dict = {T: [T*z for z in z_list] for T in T_list}
# P_list = [32768, 12000]
# delta_t=100

In [ ]:
# %%pyinstrument
# b, r, sr = single_run_lsqr(S, R, T_list, model_dict, P_list, delta_t)

# Comparison of different functions for fitting ridge regressions

In [ ]:
import json
import time
import numpy as np
import pandas as pd
from scipy.linalg import svd
from sklearn.linear_model import Ridge
import sys

sys.path.append('../models')
from ridge_solvers import ridgesvd, get_beta

from sklearn import set_config
set_config(assume_finite=True)  # up to 10% speedup

from pyinstrument import Profiler
%load_ext pyinstrument


In [ ]:
# Example usage:
T = 100
P = 1000
X = np.random.randn(T, P)
Y = np.random.randn(T)
lambd = np.array([1])

B_ridgesvd = ridgesvd(Y, X, lambd).flatten()
B_get_beta = get_beta(Y, X, lambd/T).flatten()
B_sklearn = np.linalg.inv(X.T @ X + lambd[0] * np.eye(P)) @ X.T @ Y
B_sklearn_svd = Ridge(alpha=lambd[0], fit_intercept=False, solver="svd").fit(X, Y).coef_
B_sklearn_auto = Ridge(alpha=lambd[0], fit_intercept=False, solver="auto").fit(X, Y).coef_
B_sklearn_cholesky = Ridge(alpha=lambd[0], fit_intercept=False, solver="cholesky").fit(X, Y).coef_


ref = B_ridgesvd
print(np.sum(abs(ref - B_get_beta)))
print(np.sum(abs(ref - B_ridgesvd)))
print(np.sum(abs(ref - B_sklearn)))
print(np.sum(abs(ref - B_sklearn_svd)))
print(np.sum(abs(ref - B_sklearn_auto)))
print(np.sum(abs(ref - B_sklearn_cholesky)))

3.2506332695025897e-14
0.0
9.43609697757336e-12
1.999355542237069e-14
1.6554899812115664e-14
1.6554899812115664e-14


In [ ]:
%timeit ridgesvd(Y, X, lambd).flatten()
%timeit get_beta(Y, X, lambd/T).flatten()
%timeit np.linalg.inv(X.T @ X + lambd[0] * np.eye(P)) @ X.T @ Y
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="svd").fit(X, Y).coef_
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="auto").fit(X, Y).coef_
%timeit Ridge(alpha=lambd[0], fit_intercept=False, solver="cholesky").fit(X, Y).coef_


81.3 ms ± 32.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.8 ms ± 7.87 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
112 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
42.8 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.81 ms ± 33.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.8 ms ± 58.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Standard deviations are relatively large for the first three functions, probably because something is being cached. 

# Profiling VOC and sklearn solvers

In [ ]:
import sys 
sys.path.append('../models')

In [ ]:
from voc_simulation_sklearn import *

In [ ]:
%%pyinstrument

path_to_processed = "../data/processed" # specify this
path_to_outputs = "../data/interim/simulation_outputs_sklrn_ridge" # specify this

run_simulation(999, path_to_processed, path_to_outputs)


seed: 999
Execution time: 23.561384201049805 seconds


_     ._   __/__   _ _  _  _ _/_   Recorded: 22:42:40  Samples:  9684
 /_//_/// /_\ / //_// / //_'/ //     Duration: 24.155    CPU time: 110.553
/   _/                      v4.6.2

Program: /home/miroslav/miniforge3/envs/voc/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/miroslav/.local/share/jupyter/runtime/kernel-v2-1864467guvKwoEN3YI.json

24.155 <module>  ../../../../../../../tmp/ipykernel_187651/1909041976.py:1
`- 24.152 run_simulation  voc_simulation_sklearn.py:142
   |- 23.559 single_run  voc_simulation_sklearn.py:43
   |  |- 19.888 wrapper  sklearn/base.py:1456
   |  |     [38 frames hidden]  sklearn, scipy, <built-in>, inspect
   |  |        9.627 safe_sparse_dot  sklearn/utils/extmath.py:164
   |  |        `- 9.601 [self]  sklearn/utils/extmath.py
   |  |- 3.074 Ridge.predict  sklearn/linear_model/_base.py:292
   |  |     [5 frames hidden]  sklearn
   |  `- 0.362 standardize  voc_simulation_sklearn.py:90
   |     `- 0.285 my_std  voc_simulation_sklearn.py:87
   `- 0.521 make_rff  voc_simulation_sklearn.py:10
      `- 0.496 [self]  voc_simulation_sklearn.py

In [ ]:
from voc_simulation import *

In [ ]:
%%pyinstrument

path_to_processed = "../data/processed" # specify this
path_to_outputs = "../data/interim/simulation_outputs_voc_solver" # specify this

run_simulation(999, path_to_processed, path_to_outputs)

seed: 999
Execution time: 32.91383767127991 seconds


_     ._   __/__   _ _  _  _ _/_   Recorded: 22:46:17  Samples:  1769
 /_//_/// /_\ / //_// / //_'/ //     Duration: 33.473    CPU time: 172.004
/   _/                      v4.6.2

Program: /home/miroslav/miniforge3/envs/voc/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/miroslav/.local/share/jupyter/runtime/kernel-v2-1864467guvKwoEN3YI.json

33.473 <module>  ../../../../../../../tmp/ipykernel_187651/3122287581.py:1
`- 33.471 run_simulation  voc_simulation.py:143
   |- 32.913 single_run  voc_simulation.py:46
   |  |- 27.768 get_beta  ridge_solvers.py:43
   |  |  |- 19.464 [self]  ridge_solvers.py
   |  |  `- 8.012 svd  scipy/linalg/_decomp_svd.py:13
   |  |        [2 frames hidden]  scipy
   |  |           7.979 [self]  scipy/linalg/_decomp_svd.py
   |  |- 4.622 ridgesvd  ridge_solvers.py:4
   |  |  |- 3.383 svd  scipy/linalg/_decomp_svd.py:13
   |  |  |     [2 frames hidden]  scipy
   |  |  `- 1.172 [self]  ridge_solvers.py
   |  `- 0.393 standardize  voc_simulation.py:91
   `- 0.490 make_rff  voc_simulation.py:13
      `- 0.463 [self]  voc_simulation.py

In [ ]:
%%pyinstrument

path_to_processed = "../data/processed" # specify this
path_to_outputs = "../data/interim/simulation_outputs_voc_solver" # specify this

run_simulation(999, path_to_processed, path_to_outputs)

seed: 999
Execution time: 32.91383767127991 seconds


_     ._   __/__   _ _  _  _ _/_   Recorded: 22:46:17  Samples:  1769
 /_//_/// /_\ / //_// / //_'/ //     Duration: 33.473    CPU time: 172.004
/   _/                      v4.6.2

Program: /home/miroslav/miniforge3/envs/voc/lib/python3.12/site-packages/ipykernel_launcher.py --f=/home/miroslav/.local/share/jupyter/runtime/kernel-v2-1864467guvKwoEN3YI.json

33.473 <module>  ../../../../../../../tmp/ipykernel_187651/3122287581.py:1
`- 33.471 run_simulation  voc_simulation.py:143
   |- 32.913 single_run  voc_simulation.py:46
   |  |- 27.768 get_beta  ridge_solvers.py:43
   |  |  |- 19.464 [self]  ridge_solvers.py
   |  |  `- 8.012 svd  scipy/linalg/_decomp_svd.py:13
   |  |        [2 frames hidden]  scipy
   |  |           7.979 [self]  scipy/linalg/_decomp_svd.py
   |  |- 4.622 ridgesvd  ridge_solvers.py:4
   |  |  |- 3.383 svd  scipy/linalg/_decomp_svd.py:13
   |  |  |     [2 frames hidden]  scipy
   |  |  `- 1.172 [self]  ridge_solvers.py
   |  `- 0.393 standardize  voc_simulation.py:91
   `- 0.490 make_rff  voc_simulation.py:13
      `- 0.463 [self]  voc_simulation.py